# Vectors

In [1]:
from ipynb.fs.full.koselleck import *

In [18]:
# dfmodels=get_dfmodels()
# dfmodels[dfmodels.period=='1920-1940']

In [19]:
def do_ingest_vecs(row,only_valid_words=True):
    mpath=row.path
    m=load_model(mpath)
    data=m.wv.vectors
    keys=[m.wv.index_to_key[i] for i in range(len(data))]
    res=pd.DataFrame(data, index=keys)
    if only_valid_words:
        vwords=set(get_valid_words())
        res=res.loc[set(res.index) & vwords]
    return res


In [20]:
# do_ingest_vecs(dfmodels.iloc[0])

In [21]:

def ingest_vecs(dfmodels=None, only_valid_words=True,num_proc=1,force=False):
    if dfmodels is None:
        dfmodels_all = get_pathdf_models(period_len=None)
        dfmodels_all['run_int'] = dfmodels_all['run'].apply(lambda x: int(x.split('_')[-1]))
        dfmodels=dfmodels_all[dfmodels_all.period_len.isin(PERIOD_LENS)].query(f'run_int<={MAX_RUNS}')
        # dfmodels
    with get_db('vecs',mode='a') as db:
        df=dfmodels.assign(qstr=[
            f'{row.corpus}/{row.period}/{row.run_int}'
            for i,row in dfmodels.iterrows()
        ])
        done=set(db.keys())
        if not force: df=df[~df.qstr.isin(done)]
        objs=[row for i,row in df.iterrows()]
        iterr=pmap_iter(do_ingest_vecs, objs, num_proc=num_proc)
        for i,(row,qdf) in enumerate(zip(objs,iterr)):
            db[row.qstr]=qdf
            if i and not i%10: db.commit()
        db.commit()

In [22]:
# !rm /home/ryan/db/db.kos2.vecs.sqlite
# ingest_vecs(dfmodels,num_proc=4)

In [23]:
def vecs(period,run=1,corpus=DEFAULT_CORPUS):
    qstr=f'/vecs/{corpus}/{period}/{run}'
    return dbget(qstr,pd.DataFrame())

In [25]:
# vecs('1920-1925')